# Data Aggregation
In this notebook, I will aggregate the PERSON, VEHICLE, ACCIDENT_EVENT with ACCIDENT_NO. Try to find the meaningful aggregated features to fit in the model

In [19]:
import pandas as pd
import numpy as np

# Load the all the dataset from Dataset folder
def load_dataset(file_name):
    return pd.read_csv(f'Dataset/{file_name}', low_memory=False)

file_list = [
    'ACCIDENT.csv',
    'ACCIDENT_CHAINAGE.csv',
    'ACCIDENT_EVENT.csv',
    'ACCIDENT_LOCATION.csv',
    'ATMOSPHERIC_COND.csv',
    'NODE.csv',
    'NODE_ID_COMPLEX_INT_ID.csv',
    'PERSON.csv',
    'ROAD_SURFACE_COND.csv',
    'SUBDCA.csv',
    'VEHICLE.csv'
]
# Load all datasets into a dictionary
datasets = {file_name.split('.')[0]: load_dataset(file_name) for file_name in file_list}
# Save the datasets to a dictionary for further processing
datasets = {
    'ACCIDENT': load_dataset('ACCIDENT.csv'),
    'ACCIDENT_CHAINAGE': load_dataset('ACCIDENT_CHAINAGE.csv'),
    'ACCIDENT_EVENT': load_dataset('ACCIDENT_EVENT.csv'),
    'ACCIDENT_LOCATION': load_dataset('ACCIDENT_LOCATION.csv'),
    'ATMOSPHERIC_COND': load_dataset('ATMOSPHERIC_COND.csv'),
    'NODE': load_dataset('NODE.csv'),
    'NODE_ID_COMPLEX_INT_ID': load_dataset('NODE_ID_COMPLEX_INT_ID.csv'),
    'PERSON': load_dataset('PERSON.csv'),
    'ROAD_SURFACE_COND': load_dataset('ROAD_SURFACE_COND.csv'),
    'SUBDCA': load_dataset('SUBDCA.csv'),
    'VEHICLE': load_dataset('VEHICLE.csv')
}

# Aggregation on Person dataset

In [9]:
person = datasets['PERSON'].copy()
# Keep the column with numeric values
person["AGE"] = pd.to_numeric(person["AGE"], errors="coerce")
rut_num = pd.to_numeric(person.get("ROAD_USER_TYPE"), errors="coerce")
# Masks
is_male   = person.get("SEX", pd.Series(index=person.index)).eq("M")
is_female = person.get("SEX", pd.Series(index=person.index)).eq("F")

# ROAD_USER_TYPE: 1 Pedestrian, 2 Driver, 3 Passenger, 4 Motorcyclist, 5 Pillion,
# 6 Bicyclist, 7 Other driver, 8 Other passenger, 9 Not known
m_ped, m_driver, m_passenger      = rut_num.eq(1), rut_num.eq(2), rut_num.eq(3)
m_motorcyclist, m_pillion         = rut_num.eq(4), rut_num.eq(5)
m_bicyclist, m_other_driver       = rut_num.eq(6), rut_num.eq(7)
m_other_passenger, m_user_unknown = rut_num.eq(8), rut_num.eq(9)
m_operator = rut_num.isin([2, 4, 6, 7])

tmp = person.copy()
tmp["MALE_COUNT"]            = is_male.astype(int)
tmp["FEMALE_COUNT"]          = is_female.astype(int)
tmp["N_DRIVERS"]             = m_driver.astype(int)
tmp["N_PASSENGERS"]          = m_passenger.astype(int)
tmp["N_PEDESTRIANS"]         = m_ped.astype(int)
tmp["N_MOTORCYCLISTS"]       = m_motorcyclist.astype(int)
tmp["N_PILLION"]             = m_pillion.astype(int)
tmp["N_BICYCLISTS"]          = m_bicyclist.astype(int)
tmp["N_OTHER_DRIVERS"]       = m_other_driver.astype(int)
tmp["N_OTHER_PASSENGERS"]    = m_other_passenger.astype(int)
tmp["N_USER_UNKNOWN"]        = m_user_unknown.astype(int)
# Only consider the operator's age
tmp["OPERATOR_AGE"] = np.where(m_operator, person["AGE"], np.nan)

# Aggregate to accident no
person_agg = (
    tmp.groupby("ACCIDENT_NO")
    .agg(
        PERSON_COUNT=("PERSON_ID", "count"),
        MALE_COUNT=("MALE_COUNT", "sum"),
        FEMALE_COUNT=("FEMALE_COUNT", "sum"),
        OPERATOR_MEAN_AGE=("OPERATOR_AGE", "mean"),
        OPERATOR_MEDIAN_AGE=("OPERATOR_AGE", "median"),
        N_DRIVERS=("N_DRIVERS", "sum"),
        N_PASSENGERS=("N_PASSENGERS", "sum"),
        N_PEDESTRIANS=("N_PEDESTRIANS", "sum"),
        N_MOTORCYCLISTS=("N_MOTORCYCLISTS", "sum"),
        N_PILLION=("N_PILLION", "sum"),
        N_BICYCLISTS=("N_BICYCLISTS", "sum"),
        N_OTHER_DRIVERS=("N_OTHER_DRIVERS", "sum"),
        N_OTHER_PASSENGERS=("N_OTHER_PASSENGERS", "sum"),
        N_USER_UNKNOWN=("N_USER_UNKNOWN", "sum"),
    )
    .reset_index()
)

# Derived rates
person_agg["PCT_MALE"] = (person_agg["MALE_COUNT"] / person_agg["PERSON_COUNT"]).replace([np.inf, -np.inf], np.nan)
person_agg["PCT_FEMALE"] = (person_agg["FEMALE_COUNT"] / person_agg["PERSON_COUNT"]).replace([np.inf, -np.inf], np.nan)

In [10]:
person_agg.head()

,ACCIDENT_NO,PERSON_COUNT,MALE_COUNT,FEMALE_COUNT,OPERATOR_MEAN_AGE,OPERATOR_MEDIAN_AGE,N_DRIVERS,N_PASSENGERS,N_PEDESTRIANS,N_MOTORCYCLISTS,N_PILLION,N_BICYCLISTS,N_OTHER_DRIVERS,N_OTHER_PASSENGERS,N_USER_UNKNOWN,PCT_MALE,PCT_FEMALE
0,T20060000010,6,4,2,57.666667,62.0,3,3,0,0,0,0,0,0,0,0.666667,0.333333
1,T20060000018,4,2,2,25.000000,25.0,2,2,0,0,0,0,0,0,0,0.500000,0.500000
2,T20060000022,2,1,1,45.000000,45.0,0,0,0,1,1,0,0,0,0,0.500000,0.500000
3,T20060000023,2,2,0,42.500000,42.5,2,0,0,0,0,0,0,0,0,1.000000,0.000000
4,T20060000026,3,2,1,67.500000,67.5,2,1,0,0,0,0,0,0,0,0.666667,0.333333


In [11]:
person_agg.shape

(203708, 17)

In [7]:
# ---- SAVE THE RESULT ----- if you wanna save, uncommented this cell.
# Only keep columns ACCIDENT_ID, OPERATOR_MEDIAN_AGE 
# person_agg = person_agg[["ACCIDENT_NO", "OPERATOR_MEDIAN_AGE"]]
# Save person_agg to a CSV file
# person_agg.to_csv("Clean-data/PERSON_agg_by_ACCIDENT.csv", index=False)

In [12]:
missing_values = person_agg.isnull().sum()
missing_values

ACCIDENT_NO               0
PERSON_COUNT              0
MALE_COUNT                0
FEMALE_COUNT              0
OPERATOR_MEAN_AGE      4533
OPERATOR_MEDIAN_AGE    4533
N_DRIVERS                 0
N_PASSENGERS              0
N_PEDESTRIANS             0
N_MOTORCYCLISTS           0
N_PILLION                 0
N_BICYCLISTS              0
N_OTHER_DRIVERS           0
N_OTHER_PASSENGERS        0
N_USER_UNKNOWN            0
PCT_MALE                  0
PCT_FEMALE                0
dtype: int64

In [13]:
# Mask for operator presence
has_operator = person_agg["N_DRIVERS"] + person_agg["N_MOTORCYCLISTS"] + \
               person_agg["N_BICYCLISTS"] + person_agg["N_OTHER_DRIVERS"] > 0

no_operator = person_agg.loc[person_agg["OPERATOR_MEAN_AGE"].isna() & (~has_operator)]
operator_no_age = person_agg.loc[person_agg["OPERATOR_MEAN_AGE"].isna() & has_operator]

print("No operator at all:", no_operator.shape[0])
print("Operator(s) present but all missing age:", operator_no_age.shape[0])

No operator at all: 1932
Operator(s) present but all missing age: 2601


# Aggregation on VEHICLE dataset

In [15]:
vehicle = datasets["VEHICLE"].copy()
# Keep columns with numeric values
vehicle["VEHICLE_TYPE"]      = pd.to_numeric(vehicle["VEHICLE_TYPE"], errors="coerce")
vehicle["SEATING_CAPACITY"]  = pd.to_numeric(vehicle["SEATING_CAPACITY"], errors="coerce")
vehicle["NO_OF_WHEELS"]      = pd.to_numeric(vehicle["NO_OF_WHEELS"], errors="coerce")
vehicle["TARE_WEIGHT"]       = pd.to_numeric(vehicle["TARE_WEIGHT"], errors="coerce")
vehicle["TOTAL_NO_OCCUPANTS"]= pd.to_numeric(vehicle["TOTAL_NO_OCCUPANTS"], errors="coerce")
vehicle["LEVEL_OF_DAMAGE"]   = pd.to_numeric(vehicle["LEVEL_OF_DAMAGE"], errors="coerce")
vehicle["TOWED_AWAY_FLAG"]   = pd.to_numeric(vehicle["TOWED_AWAY_FLAG"], errors="coerce")
vehicle["CAUGHT_FIRE"]       = pd.to_numeric(vehicle["CAUGHT_FIRE"], errors="coerce")

# Impute by VEHICLE_TYPE, fill the NaN with median values of this column
cols_to_impute = ["SEATING_CAPACITY", "NO_OF_WHEELS", "TARE_WEIGHT"]

for col in cols_to_impute:
    type_median = vehicle.groupby("VEHICLE_TYPE")[col].transform("median")
    vehicle[f"{col}_IMP"] = vehicle[col].fillna(type_median)
    global_med = vehicle[col].median(skipna=True)
    vehicle[f"{col}_IMP"] = vehicle[f"{col}_IMP"].fillna(global_med)

# TOWED_AWAY_FLAG: 1 Yes, 2 No, 9 Unknown
vehicle["TOWED_YES"]   = (vehicle["TOWED_AWAY_FLAG"] == 1).astype(int)
vehicle["TOWED_KNOWN"] = vehicle["TOWED_AWAY_FLAG"].isin([1, 2]).astype(int)

# CAUGHT_FIRE: 0 N/A, 1 Yes, 2 No, 9 Unknown
vehicle["FIRE_YES"]    = (vehicle["CAUGHT_FIRE"] == 1).astype(int)
vehicle["FIRE_KNOWN"]  = vehicle["CAUGHT_FIRE"].isin([1, 2]).astype(int)

# DAMAGE: 1..6 valid, 9 unknown; treat severe = {4,5}
vehicle["SEVERE_DAMAGE"] = vehicle["LEVEL_OF_DAMAGE"].isin([4, 5]).astype(int)
vehicle["DAMAGE_KNOWN"]  = vehicle["LEVEL_OF_DAMAGE"].isin([1,2,3,4,5,6]).astype(int)
vehicle["N_VEHICLES"] = 1  # per row = a vehicle
vehicle_agg = (
    vehicle.groupby("ACCIDENT_NO")
    .agg(
        # imputed numeric features with mean and median value of this column
        MEAN_SEATING=("SEATING_CAPACITY_IMP","mean"),
        MEDIAN_SEATING=("SEATING_CAPACITY_IMP","median"),
        MEAN_WHEELS=("NO_OF_WHEELS_IMP","mean"),
        MEDIAN_WHEELS=("NO_OF_WHEELS_IMP","median"),
        SUM_TARE=("TARE_WEIGHT_IMP","sum"),
        MEAN_TARE=("TARE_WEIGHT_IMP","mean"),
        MEAN_YEAR_MANUF=("VEHICLE_YEAR_MANUF","mean"),
        MIN_YEAR_MANUF=("VEHICLE_YEAR_MANUF","min"),
        MAX_YEAR_MANUF=("VEHICLE_YEAR_MANUF","max"),
        MEAN_OCCUPANTS=("TOTAL_NO_OCCUPANTS","mean"),
        N_TOWED=("TOWED_YES","sum"),
        N_TOWED_KNOWN=("TOWED_KNOWN","sum"),
        N_FIRE=("FIRE_YES","sum"),
        N_FIRE_KNOWN=("FIRE_KNOWN","sum"),
        N_SEVERE_DAMAGE=("SEVERE_DAMAGE","sum"),
        N_DAMAGE_KNOWN=("DAMAGE_KNOWN","sum"),
    )
    .reset_index()
)


In [16]:
missing_values = vehicle_agg.isnull().sum()
missing_values

ACCIDENT_NO           0
MEAN_SEATING          0
MEDIAN_SEATING        0
MEAN_WHEELS           0
MEDIAN_WHEELS         0
SUM_TARE              0
MEAN_TARE             0
MEAN_YEAR_MANUF    5990
MIN_YEAR_MANUF     5990
MAX_YEAR_MANUF     5990
MEAN_OCCUPANTS        0
N_TOWED               0
N_TOWED_KNOWN         0
N_FIRE                0
N_FIRE_KNOWN          0
N_SEVERE_DAMAGE       0
N_DAMAGE_KNOWN        0
dtype: int64

In [17]:
vehicle_agg.describe()

,MEAN_SEATING,MEDIAN_SEATING,MEAN_WHEELS,MEDIAN_WHEELS,SUM_TARE,MEAN_TARE,MEAN_YEAR_MANUF,MIN_YEAR_MANUF,MAX_YEAR_MANUF,MEAN_OCCUPANTS,N_TOWED,N_TOWED_KNOWN,N_FIRE,N_FIRE_KNOWN,N_SEVERE_DAMAGE,N_DAMAGE_KNOWN
count,203708.000000,203708.000000,203708.000000,203708.000000,203708.000000,203708.000000,197718.000000,197718.000000,197718.000000,203708.000000,203708.000000,203708.000000,203708.000000,203708.000000,203708.000000,203708.000000
mean,4.424697,4.423707,3.619165,3.618847,2621.479770,1424.329580,1910.063062,1851.328316,1965.619711,1.298170,0.835809,1.691033,0.005827,1.665772,0.558677,1.678304
std,2.801599,2.778930,0.892949,0.894703,2667.547422,1385.410793,350.919690,525.440146,279.128467,0.728597,0.874040,0.801877,0.079765,0.794800,0.735548,0.806491
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.500000,3.500000,4.000000,4.000000,1300.000000,892.500000,1997.000000,1994.000000,2001.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,5.000000,5.000000,4.000000,4.000000,2050.000000,1302.000000,2003.000000,2000.000000,2006.000000,1.000000,1.000000,2.000000,0.000000,2.000000,0.000000,2.000000
75%,5.000000,5.000000,4.000000,4.000000,3047.000000,1546.500000,2007.333333,2006.000000,2010.000000,1.500000,1.000000,2.000000,0.000000,2.000000,1.000000,2.000000
max,65.000000,65.000000,32.500000,32.500000,106280.000000,53140.000000,2502.500000,2020.000000,3001.000000,48.500000,10.000000,21.000000,5.000000,21.000000,7.000000,19.000000


# Aggregation on ACCIDENT_EVENT dataset

In [18]:
events = datasets['ACCIDENT_EVENT'].copy()
events_sorted = events.sort_values(["ACCIDENT_NO", "EVENT_SEQ_NO"])
# Get the first event occured
first_event = (
    events[events["EVENT_SEQ_NO"] == 1]
    [["ACCIDENT_NO", "Event Type Desc"]]
)
first_event.shape

(203648, 2)